In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [15]:
def load_data():
    df = pd.read_csv("All_Streaming_Shows.csv")
    df = df[['Series Title', 'Genre', 'Description', 'No of Seasons', 'Streaming Platform']]
    df = df.dropna()
    return df

In [16]:
def content_based_recommendations(df, title):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df['Genre'] + ' ' + df['Description'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    indices = pd.Series(df.index, index=df['Series Title'])
    if title not in indices:
        return "Title not found in dataset."
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:4]
    series_indices = [i[0] for i in sim_scores]
    recommendations = df.iloc[series_indices]
    return recommendations[['Series Title', 'No of Seasons', 'Streaming Platform']]


In [17]:
def collaborative_filtering():
    data = {
        'user_id': [1, 1, 2, 2, 3, 3],
        'title': ['Breaking Bad', 'Dark', 'Game of Thrones', 'Money Heist', 'The Witcher', 'Dark'],
        'rating': [5, 4, 5, 4, 3, 5]
    }
    df = pd.DataFrame(data)
    reader = Reader(rating_scale=(1, 5))
    dataset = Dataset.load_from_df(df[['user_id', 'title', 'rating']], reader)
    trainset, testset = train_test_split(dataset, test_size=0.2)
    model = SVD()
    model.fit(trainset)
    
    predictions = model.test(testset)
    return predictions

In [18]:
df = load_data()
print("Content-Based Recommendations for 'Dark':")
print(content_based_recommendations(df, 'Dark'))

print("Collaborative Filtering Predictions:")
print(collaborative_filtering())


Content-Based Recommendations for 'Dark':
         Series Title No of Seasons Streaming Platform
633              Safe       1Season            Netflix
706  Home Before Dark       1Season          Apple TV+
423       The Missing      2Seasons  Prime Video,Starz
Collaborative Filtering Predictions:
[Prediction(uid=1, iid='Breaking Bad', r_ui=5.0, est=4.001455154014174, details={'was_impossible': False}), Prediction(uid=3, iid='Dark', r_ui=5.0, est=3.6572394185336714, details={'was_impossible': False})]
